# Pull data and store in MongoDB

## Get location info

In [480]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [481]:
def get_stations(soup):
    stations = []
    for ele in soup.find_all(class_ = 'stationname'):
        stations.append(ele.find('a').string)
    return stations

def get_station_ids(soup):
    station_ids = []
    for ele in soup.find_all(class_ = 'stationid'):
        station_ids.append(ele.text)
    return station_ids
    
def get_lats(soup):
    lats = []
    for ele in soup.find_all(class_ = 'latitude'):
        lats.append(ele.text)
        lats = [lat.replace("+","") for lat in lats]
    return lats

def get_longs(soup):
    longs = []
    for ele in soup.find_all(class_ = 'longitude'):
        longs.append(ele.text)
    return longs

In [482]:
import time
from bs4 import BeautifulSoup
import requests
url_list = ['https://tidesandcurrents.noaa.gov/tide_predictions.html?gid=1393#listing', 
            'https://tidesandcurrents.noaa.gov/tide_predictions.html?gid=1409#listing',
           'https://tidesandcurrents.noaa.gov/tide_predictions.html?gid=1415#listing']
station_dict = {}
for url in url_list:
    driver.get(url)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    station_dict['station_name'] = get_stations(soup)
    station_dict['station_id'] = get_station_ids(soup)
    station_dict['lat'] = get_lats(soup)
    station_dict['long'] = get_longs(soup)
    driver.execute_script("window.stop();")

In [483]:
assert len(station_dict['station_name'])==len(station_dict['station_id']) == \
len(station_dict['lat']) == len(station_dict['long']), \
f"Lengths of dictionary elements differ: \n\
Num Station Names: {len(station_dict['station_name'])} \n\
Num Station Ids: {len(station_dict['station_id'])} \n\
Num Latitudes: {len(station_dict['lat'])} \n\
Num Longitudes: {len(station_dict['long'])}"

In [509]:
stations = []
for idx, val in enumerate(station_dict['station_name']):
    dict_to_append = {'station_name':station_dict['station_name'][idx],
                      'station_id':station_dict['station_id'][idx],
                      'lat':station_dict['lat'][idx],
                      'long':station_dict['long'][idx]
                      }
    stations.append(dict_to_append)

## Get weather data and put into MongoDB

In [510]:
#get weather data for the next 72 hours
import requests
def get_weather_data(stations):
    weather_list = []
    weather_url = "https://weatherbit-v1-mashape.p.rapidapi.com/forecast/hourly"
    for station in stations:
        querystring = {"lat":station['lat'],"lon":station['long'],"hours":"72"}
        headers = {
            "X-RapidAPI-Key": "1282a0348bmsh6030279122a830ap16b70fjsn3c47099439d6",
            "X-RapidAPI-Host": "weatherbit-v1-mashape.p.rapidapi.com"
        }
        weather_dict = {'station_name':station['station_name'],
                        'station_id':station['station_id'],
                        'lat':station['lat'],
                        'long':station['long'],
                        'response':requests.request("GET", weather_url, headers=headers, params=querystring).json()['data']
                       }
        weather_list.append(weather_dict)
    return weather_list

In [511]:
weather_data = get_weather_data(stations)

In [ ]:
# !mongod --dbpath ~/data/paddleboard_db

In [513]:
!brew services start mongodb/brew/mongodb-community

==> Successfully started `mongodb-community` (label: homebrew.mxcl.mongodb-commu


In [514]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

In [515]:
client.list_database_names()

['admin', 'config', 'local']

In [516]:
db = client["paddleboard"]
weather = db["weather"]
weather.insert_many(weather_data)

## Get tide data and put into MongoDB

In [496]:
def get_tides_data(stations):
    tides_response = []
    for station in stations:
        tides_url = "https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/{}.json?expand=products".format(station['station_id'])
        tides_dict = {'station_name':station['station_name'],
                      'station_id':station['station_id'],
                      'lat':station['lat'],
                      'long':station['long'],
                      'response':requests.request("GET", tides_url).json()
                     }
        tides_response.append(tides_dict)
    return tides_response

In [497]:
tides_data = get_tides_data(stations)

In [517]:
#put tides data in MongoDB
tides = db["tides"]
tides.insert_many(tides_data)

## Get air quality data and put into MongoDB

In [518]:
def get_air_quality_data(stations):
    aqi_list = []
    for station in stations:
        aqi_url = "http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={}&lon={}&appid={}".format(
            station['lat'], station['long'], '12b57316a4ce5ebac5e0386d9ee999be'
        )
        aqi_dict = {'station_name':station['station_name'],
                    'station_id':station['station_id'],
                    'lat':station['lat'],
                    'long':station['long'],
                    'response':requests.request("GET", aqi_url).json()
             }
        aqi_list.append(aqi_dict)
    print(aqi_url) #checking one output of the url format to make sure it's working smoothly
    return aqi_list

In [519]:
aqi_data = get_air_quality_data(stations)

http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat=48.7867&lon=-122.9700&appid=12b57316a4ce5ebac5e0386d9ee999be


In [520]:
#put AQI data in MongoDB
aqi = db["aqi"]
aqi.insert_many(aqi_data)

In [521]:
print("Databases:", client.list_database_names())
print("Collections:", db.list_collection_names())

Databases: ['admin', 'config', 'local', 'paddleboard']
Collections: ['tides', 'aqi', 'weather']


## Shut down MongoDB

In [498]:
!brew services stop mongodb/brew/mongodb-community

Stopping `mongodb-community`... (might take a while)
==> Successfully stopped `mongodb-community` (label: homebrew.mxcl.mongodb-commu


## Extras

In [571]:
test = db['tides']
cursor = test.find()
list_cur = list(cursor)

In [572]:
tides_test = list_cur

In [587]:
tides_test[0]['response']['stations'][0]['products']['products'][0]['value']

'https://tidesandcurrents.noaa.gov/noaatidepredictions.html?id=9440574'

In [588]:
tides_test[0]

{'_id': ObjectId('62f0398ac868aa58a7095033'),
 'station_name': 'Columbia River entrance (N. Jetty)',
 'station_id': '9440574',
 'lat': '46.2733',
 'long': '-124.0720',
 'response': {'count': 1,
  'units': None,
  'stations': [{'tidal': True,
    'greatlakes': False,
    'shefcode': None,
    'details': {'self': 'https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/9440574/details.json'},
    'sensors': {'self': 'https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/9440574/sensors.json'},
    'floodlevels': {'self': 'https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/9440574/floodlevels.json'},
    'datums': {'self': 'https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/9440574/datums.json'},
    'supersededdatums': {'self': 'https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/9440574/supersededdatums.json'},
    'harmonicConstituents': {'self': 'https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/9440574/harcon

In [560]:
latlong

[46.2733, -124.072]

In [557]:
aqi_test[1]

{'_id': ObjectId('62f0397dc868aa58a7094f79'),
 'station_name': 'Cape Disappointment',
 'station_id': '9440581',
 'lat': '46.2810',
 'long': '-124.0463',
 'response': [{'wind_cdir': 'N',
   'rh': 60,
   'pod': 'd',
   'timestamp_utc': '2022-08-07T23:00:00',
   'pres': 1012.5,
   'solar_rad': 682.645,
   'ozone': 308.3,
   'weather': {'icon': 'c01d', 'code': 800, 'description': 'Clear Sky'},
   'wind_gust_spd': 8.59,
   'timestamp_local': '2022-08-07T16:00:00',
   'snow_depth': 0,
   'clouds': 0,
   'ts': 1659913200,
   'wind_spd': 5.81,
   'pop': 0,
   'wind_cdir_full': 'north',
   'slp': 1014.5,
   'dni': 853.55,
   'dewpt': 15.5,
   'snow': 0,
   'uv': 5.4,
   'wind_dir': 2,
   'clouds_hi': 0,
   'precip': 0,
   'vis': 31.296,
   'dhi': 108.54,
   'app_temp': 23.8,
   'datetime': '2022-08-07:23',
   'temp': 23.8,
   'ghi': 710.51,
   'clouds_mid': 0,
   'clouds_low': 0},
  {'wind_cdir': 'N',
   'rh': 55,
   'pod': 'd',
   'timestamp_utc': '2022-08-08T00:00:00',
   'pres': 1012.5,
   '

In [535]:
aqi_test['station_name'=='Columbia River entrance (N. Jetty)']

'46.2733'

In [537]:
keys = ['lat','long']
latlong = [float(aqi_data['station_name'=='Columbia River entrance (N. Jetty)'][key]) for key in keys]

In [538]:
latlong

[46.2733, -124.072]

In [527]:
[item['station_name'] for item in aqi_test]

['Columbia River entrance (N. Jetty)',
 'Cape Disappointment',
 "Fort Canby, Jetty 'A', Wash.",
 'Ilwaco, Baker Bay, Wash.',
 'Chinook, Baker Bay, Wash.',
 'Hungry Harbor, Wash.',
 'Hammond, Oregon',
 'Point Adams, Oreg.',
 'Astoria (Youngs Bay), Oreg.',
 'Cathcart Landing, Youngs River',
 'Warrenton, Skipanon River, Oreg.',
 'Astoria (Port Docks), Oreg.',
 'ASTORIA (Tongue Point), Oreg.',
 'Knappa, Knappa Slough',
 'Settlers Point, Oreg.',
 'Harrington Point, Wash.',
 'Skamokawa',
 'Wauna',
 'Cape Horn',
 'Barlow Point',
 'Longview',
 'Temco Kalama Terminal',
 'Saint Helens',
 'Knapp Landing, Willow Bar',
 'Rocky Point',
 'Portland Morrison Street Bridge',
 'Vancouver',
 'Washougal',
 'Beacon Rock State Park',
 'Nahcotta',
 'Tarlatt Slough',
 'Greenhead Slough',
 'Paradise Point, Long Island',
 'Naselle River, swing bridge',
 'Naselle River, 4 miles above swing bridge',
 'Bay Center, Palix River',
 'Palix River, south fork',
 'TOKE POINT',
 'Mailboat Slough, Willapa River',
 'South Be

In [426]:
wind = [i['wind_spd'] for i in aqi_test]
date_times = [i['datetime'].split(":") for i in aqi_test]
date = pd.to_datetime([i[0] for i in date_times])
hour = pd.Series([i[1] for i in date_times])
date_times = date + hour.astype("timedelta64[h]")
wind_dict = {'dts':date_times, 'wind':wind}
wind_df = pd.DataFrame(wind_dict)
wind_df_filtered = wind_df[wind_df.dts.dt.date == d]

In [425]:
d = '2022-08-05'

In [ ]:
pd.to_datetime(df.DATE) + df.HOUR.astype('timedelta64[h]')

In [ ]:
import datetime
today = date.today()
end_date = today + datetime.timedelta(days=3)
begin_date={today.strftime('%Y%m%d')}& \
end_date={end_date.strftime('%Y%m%d')}& \

In [271]:
dt = 1660039200
pd.to_datetime(dt, unit='s', origin='unix') 

Timestamp('2022-08-09 10:00:00')

In [123]:
#pull data from MongoDB and make a dataframe
import pandas as pd
cursor = weather.find()
list_cur = list(cursor)
weather_df = pd.DataFrame(list_cur)

In [125]:
weather_df.head()

,_id,Columbia River entrance (N. Jetty),Cape Disappointment,"Fort Canby, Jetty 'A', Wash.","Ilwaco, Baker Bay, Wash.","Chinook, Baker Bay, Wash.","Hungry Harbor, Wash.","Hammond, Oregon","Point Adams, Oreg.","Astoria (Youngs Bay), Oreg.",...,"Echo Bay, Sucia Islands",Ferndale,CHERRY POINT,"Blaine, Semiahmoo Bay","POINT ROBERTS, PUGET SOUND","Kanaka Bay, San Juan Island","Hanbury Point, Mosquito Pass, San Juan I.","Roche Harbor, San Juan Island","WALDRON ISLAND, PUGET SOUND",Patos Island Wharf
0,62ed406dc868aa58a7094f6e,"{'station_id': '9440574', 'lat': '+46.2733', '...","{'station_id': '9440581', 'lat': '+46.2810', '...","{'station_id': '9440572', 'lat': '+46.2683', '...","{'station_id': '9440597', 'lat': '+46.3033', '...","{'station_id': '9440573', 'lat': '+46.2717', '...","{'station_id': '9440563', 'lat': '+46.2583', '...","{'station_id': '9439011', 'lat': '+46.2017', '...","{'station_id': 'TWC0881', 'lat': '+46.2000', '...","{'station_id': '9439026', 'lat': '+46.1717', '...",...,"{'station_id': '9449712', 'lat': '+48.7567', '...","{'station_id': 'TWC1189', 'lat': '+48.8333', '...","{'station_id': '9449424', 'lat': '+48.8633', '...","{'station_id': '9449679', 'lat': '+48.9917', '...","{'station_id': '9449639', 'lat': '+48.9750', '...","{'station_id': '9449856', 'lat': '+48.4850', '...","{'station_id': '9449828', 'lat': '+48.5817', '...","{'station_id': '9449834', 'lat': '+48.6100', '...","{'station_id': '9449746', 'lat': '+48.6868', '...","{'station_id': '9449704', 'lat': '+48.7867', '..."


In [125]:
weather_df.head()

,_id,Columbia River entrance (N. Jetty),Cape Disappointment,"Fort Canby, Jetty 'A', Wash.","Ilwaco, Baker Bay, Wash.","Chinook, Baker Bay, Wash.","Hungry Harbor, Wash.","Hammond, Oregon","Point Adams, Oreg.","Astoria (Youngs Bay), Oreg.",...,"Echo Bay, Sucia Islands",Ferndale,CHERRY POINT,"Blaine, Semiahmoo Bay","POINT ROBERTS, PUGET SOUND","Kanaka Bay, San Juan Island","Hanbury Point, Mosquito Pass, San Juan I.","Roche Harbor, San Juan Island","WALDRON ISLAND, PUGET SOUND",Patos Island Wharf
0,62ed406dc868aa58a7094f6e,"{'station_id': '9440574', 'lat': '+46.2733', '...","{'station_id': '9440581', 'lat': '+46.2810', '...","{'station_id': '9440572', 'lat': '+46.2683', '...","{'station_id': '9440597', 'lat': '+46.3033', '...","{'station_id': '9440573', 'lat': '+46.2717', '...","{'station_id': '9440563', 'lat': '+46.2583', '...","{'station_id': '9439011', 'lat': '+46.2017', '...","{'station_id': 'TWC0881', 'lat': '+46.2000', '...","{'station_id': '9439026', 'lat': '+46.1717', '...",...,"{'station_id': '9449712', 'lat': '+48.7567', '...","{'station_id': 'TWC1189', 'lat': '+48.8333', '...","{'station_id': '9449424', 'lat': '+48.8633', '...","{'station_id': '9449679', 'lat': '+48.9917', '...","{'station_id': '9449639', 'lat': '+48.9750', '...","{'station_id': '9449856', 'lat': '+48.4850', '...","{'station_id': '9449828', 'lat': '+48.5817', '...","{'station_id': '9449834', 'lat': '+48.6100', '...","{'station_id': '9449746', 'lat': '+48.6868', '...","{'station_id': '9449704', 'lat': '+48.7867', '..."


In [126]:
#show wind, temp, clouds for the next 24 hours
weather_df[['timestamp_local','wind_spd','temp','clouds']].head(5)

KeyError: "None of [Index(['timestamp_local', 'wind_spd', 'temp', 'clouds'], dtype='object')] are in the [columns]"